In [1]:
!pip install fuzzysearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 507.0 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for fuzzysearch: filename=fuzzysearch-0.7.3-py3-none-any.whl size=21203 sha256=317c39173f78aa5d77b01459b2e91bae928ea7414550b81478a5f8ac69a73ee0
  Stored in directory: /tmp/pip-ephem-wheel-cache-vvujnwtm/wheels/be/ad/2e/bd664c4b01e5535ee4387d8a491311f61467a43627597684a7
Successfully built fuzzysearch


In [2]:
from rapidfuzz import process, fuzz
import re
from fuzzysearch import find_near_matches


In [3]:
import re

def correct_units(text:str):
    rules = [
        (r'(\d+)(n|п)\b', r'\1л'),
        (r'(\d+)r\b', r'\1г'),
        (r'(\d+)(mn|мn|mл)\b', r'\1мл'),
        (r'(\d+)(mr|мr|mг)\b', r'\1мг'),
        (r'(\d+)([yу]n|уn|уп)\b', r'\1уп'),
        (r'(\d+)(kr|кr|kг)\b', r'\1кг'),
    ]
    for pattern, repl in rules:
        text = re.sub(pattern, repl, text, flags=re.IGNORECASE)
    return text


In [4]:

class PhraseCorrectorByWords:
    def __init__(self, words_file="words.csv",score_cutoff=70, min_len = 5):
        with open(words_file, "r", encoding="utf-8") as f:
            self.reference_words = [line.strip() for line in f]
        self.score_cutoff = score_cutoff
        self.min_len = min_len





    def correct_text(self,text):
        corrected_lines = []
        corrections_log = []
        for line_num, line in enumerate(text.strip().split('\n')):
            corrected_words = []
            words = re.split(r"[ \t\f\v.,!?;:()\"«»—–]+", line.upper().replace("Ё", "Е"))
            words = list(filter(None, words))  # убираем пустые строки, если они есть
        
            for word_pos, word in enumerate(words):
                if len(word) < self.min_len:
                    corrected_words.append(word)
                    continue
                # Ищем наиболее близкое слово из словаря
                match = process.extractOne(word, self.reference_words, scorer=fuzz.ratio, score_cutoff=self.score_cutoff)
                #print(match)
                corrected_word = match[0] if match else word
                corrected_words.append(corrected_word)
                if match and corrected_word != word:
                    corrections_log.append({
                        "line": line_num + 1,
                        "position": word_pos + 1,
                        "original": word,
                        "corrected": corrected_word,
                        "score": match[1]
                    })
            corrected_lines.append(' '.join(corrected_words))
        return '\n'.join(corrected_lines), corrections_log
    


In [5]:
import re
from rapidfuzz import fuzz, process

class PhraseCorrectorNgrams:
    def __init__(self, words_file="words.csv", score_cutoff=85, min_len=3, max_ngram=3):
        # Чтение и нормализация фраз
        with open(words_file, "r", encoding="utf-8") as f:
            raw_phrases = [line.strip().upper().replace("Ё", "Е") for line in f if line.strip()]

        # Группировка по количеству слов
        self.reference_by_length = {}
        for phrase in raw_phrases:
            word_count = len(phrase.split())
            if word_count not in self.reference_by_length:
                self.reference_by_length[word_count] = []
            self.reference_by_length[word_count].append(phrase)

        self.score_cutoff = score_cutoff
        self.min_len = min_len
        self.max_ngram = max_ngram

    def correct_text(self, text):
        text = text.upper().replace("Ё", "Е")
        tokens = re.split(r"[ \n\t\f\v!?;:()\"«»]+", text)
        tokens = list(filter(None, tokens))

        corrected_tokens = tokens[:]
        corrected = [False] * len(tokens)
        corrections_log = []

        for n in range(self.max_ngram, 0, -1):
            if n not in self.reference_by_length:
                continue  # нет эталонов нужной длины

            for i in range(len(tokens) - n + 1):
                if any(corrected[i:i+n]):
                    continue

                ngram_tokens = tokens[i:i+n]
                ngram = ' '.join(ngram_tokens)
                if len(ngram) < self.min_len:
                    continue

                match = process.extractOne(
                    ngram,
                    self.reference_by_length[n],
                    scorer=fuzz.ratio,
                    score_cutoff=self.score_cutoff
                )
                if match:
                    corrected_tokens[i] = match[0]
                    for j in range(i+1, i+n):
                        corrected_tokens[j] = ''
                    corrected[i:i+n] = [True]*n

                    corrections_log.append({
                        "position": i,
                        "original": ngram,
                        "corrected": match[0],
                        "score": match[1],
                    })

        final_text = ' '.join(filter(None, corrected_tokens))
        return final_text, corrections_log


In [7]:

class PhraseCorrectorFuzzySearch:
    def __init__(self, words_file="words.csv", max_l_dist=1, min_len=5):
        with open(words_file, "r", encoding="utf-8") as f:
            self.reference_phrases = [line.strip().upper().replace("Ё", "Е") for line in f]
        self.max_l_dist = max_l_dist
        self.min_len = min_len

    def correct_text(self, text):
        text = text.upper().replace("Ё", "Е")
        corrected_text = text
        corrections_log = []

        for phrase in self.reference_phrases:
            if len(phrase) < self.min_len:
                continue

            matches = find_near_matches(phrase, corrected_text, max_l_dist=self.max_l_dist)
            for match in matches:
                original_segment = corrected_text[match.start:match.end]
                corrected_text = (
                    corrected_text[:match.start] + phrase + corrected_text[match.end:]
                )

                corrections_log.append({
                    "original": original_segment,
                    "corrected": phrase,
                    "start_pos": match.start,
                    "end_pos": match.end,
                    "dist": match.dist
                })

        return corrected_text, corrections_log


In [47]:
to_be_corerected_corpus  = []
to_be_corerected_corpus.append( '''
Пюре Фрутонята
Пюре Фруктоня
Цыпленок 80г
Цыпденок 80r
Мясое изделие для детского питания
''')
to_be_corerected_corpus.append('''
Пюре фруктоня
цыбленок 80000г
мясное для детского питания
''')
to_be_corerected_corpus.append('''
бальзам ШАУМ 7
ТРАВ 300мл 
для нормальных и жирных волос
''')
to_be_corerected_corpus.append('''
Тютюнок Занукелия
500 мг
''')
to_be_corerected_corpus.append('''
Вино АБРАУ-ДЮРОСО
0.75л
13% объёмное, белое п/с
''')



In [45]:
phrase_corrector = PhraseCorrectorByWords(words_file="words.txt", score_cutoff=75,  min_len=2)
for text in to_be_corerected_corpus:
    text = correct_units(text)
    corrected_text, corrections_log = phrase_corrector.correct_text(text)
    print("Original Text:\n", text)
    print("Corrected Text:\n", corrected_text)
    for entry in corrections_log:
        print(entry)
    print("-" * 50)

Original Text:
 
Пюре Фрутонята
Цыпленок 80г
Цыпденок 80г
Мясое изделие для детского питания

Corrected Text:
 ПЮРЕ ФРУТОНЯНЯ
ЦЫПЛЕНОК 80Г
ЦЫПЛЕНОК 80Г
МЯСНОЕ ИЗДЕЛИЕ ДЛЯ ДЕТСКОГО ПИТАНИЯ
{'line': 1, 'position': 2, 'original': 'ФРУТОНЯТА', 'corrected': 'ФРУТОНЯНЯ', 'score': 77.77777777777779}
{'line': 3, 'position': 1, 'original': 'ЦЫПДЕНОК', 'corrected': 'ЦЫПЛЕНОК', 'score': 87.5}
{'line': 4, 'position': 1, 'original': 'МЯСОЕ', 'corrected': 'МЯСНОЕ', 'score': 90.9090909090909}
--------------------------------------------------
Original Text:
 
Пюре фруктоня
цыбленок 80000г
мясное для детского питания

Corrected Text:
 ПЮРЕ ФРУТОНЯНЯ
ЦЫПЛЕНОК 80000Г
МЯСНОЕ ДЛЯ ДЕТСКОГО ПИТАНИЯ
{'line': 1, 'position': 2, 'original': 'ФРУКТОНЯ', 'corrected': 'ФРУТОНЯНЯ', 'score': 82.35294117647058}
{'line': 2, 'position': 1, 'original': 'ЦЫБЛЕНОК', 'corrected': 'ЦЫПЛЕНОК', 'score': 87.5}
--------------------------------------------------
Original Text:
 
бальзам ШАУМ 7
ТРАВ 300мл 
для нормальных и жирных

In [53]:
phrase_corrector = PhraseCorrectorNgrams(words_file="words.txt", score_cutoff=75,  min_len=2)
for text in to_be_corerected_corpus:
    text = correct_units(text)
    corrected_text, corrections_log = phrase_corrector.correct_text(text)
    print("Original Text:\n", text)
    print("Corrected Text:\n", corrected_text)
    for entry in corrections_log:
        print(entry)
    print("-" * 50)

Original Text:
 
Пюре Фрутонята
Пюре Фруктоня
Цыпленок 80г
Цыпденок 80г
Мясое изделие для детского питания

Corrected Text:
 ПЮРЕ ФРУТОНЯНЯ ПЮРЕ ФРУТОНЯНЯ ЦЫПЛЕНОК 80Г ЦЫПЛЕНОК 80Г МЯСНОЕ ИЗДЕЛИЕ ДЛЯ ДЕТСКОГО ПИТАНИЯ
{'position': 0, 'original': 'ПЮРЕ', 'corrected': 'ПЮРЕ', 'score': 100.0}
{'position': 1, 'original': 'ФРУТОНЯТА', 'corrected': 'ФРУТОНЯНЯ', 'score': 77.77777777777779}
{'position': 2, 'original': 'ПЮРЕ', 'corrected': 'ПЮРЕ', 'score': 100.0}
{'position': 3, 'original': 'ФРУКТОНЯ', 'corrected': 'ФРУТОНЯНЯ', 'score': 82.35294117647058}
{'position': 4, 'original': 'ЦЫПЛЕНОК', 'corrected': 'ЦЫПЛЕНОК', 'score': 100.0}
{'position': 6, 'original': 'ЦЫПДЕНОК', 'corrected': 'ЦЫПЛЕНОК', 'score': 87.5}
{'position': 8, 'original': 'МЯСОЕ', 'corrected': 'МЯСНОЕ', 'score': 90.9090909090909}
{'position': 9, 'original': 'ИЗДЕЛИЕ', 'corrected': 'ИЗДЕЛИЕ', 'score': 100.0}
{'position': 11, 'original': 'ДЕТСКОГО', 'corrected': 'ДЕТСКОГО', 'score': 100.0}
{'position': 12, 'original': 'ПИТАНИЯ',

In [37]:
phrase_corrector = PhraseCorrectorFuzzySearch(words_file="words.txt")
for text in to_be_corerected_corpus:
    text = correct_units(text)
    corrected_text, corrections_log = phrase_corrector.correct_text(text)
    print("Original Text:\n", text)
    print("Corrected Text:\n", corrected_text)
    for entry in corrections_log:
        print(entry)
    print("-" * 50)

Original Text:
 
Пюре Фрутонята
Цыпленок 80г
Цыпденок 80г
Мясое изделие для детского питания

Corrected Text:
 
ПЮРЕ ФРУТИПЯТКИ
ЦЫПЛЩЕНОКИ80Г
ЦЫПЩЕНОКИ 80Г
МЯСОМ ИЗДЕЛИЕ ДЛЯ ДЕТСКИЕО ПИНТАНИЯ

{'original': 'ЦЫПЛЕНОК', 'corrected': 'ЦЫПЛЕНОК', 'start_pos': 16, 'end_pos': 24, 'dist': 0}
{'original': 'ЦЫПДЕНОК', 'corrected': 'ЦЫПЛЕНОК', 'start_pos': 29, 'end_pos': 37, 'dist': 1}
{'original': 'ЦЫПЛЕНОК', 'corrected': 'ЦЫПЛЕНОК', 'start_pos': 16, 'end_pos': 24, 'dist': 0}
{'original': 'ЦЫПЛЕНОК', 'corrected': 'ЦЫПЛЕНОК', 'start_pos': 29, 'end_pos': 37, 'dist': 0}
{'original': 'ДЕТСКОГ', 'corrected': 'ДЕТСКОЕ', 'start_pos': 60, 'end_pos': 67, 'dist': 1}
{'original': 'ИЗДЕЛИЕ', 'corrected': 'ИЗДЕЛИЯ', 'start_pos': 48, 'end_pos': 55, 'dist': 1}
{'original': 'ФРУТОН', 'corrected': 'ФРУТО-', 'start_pos': 6, 'end_pos': 12, 'dist': 1}
{'original': 'ФРУТО', 'corrected': 'ФРУТО', 'start_pos': 6, 'end_pos': 11, 'dist': 0}
{'original': 'ИЗДЕЛИЯ', 'corrected': 'ИЗДЕЛИЯ', 'start_pos': 48, 'end_pos': 55,

In [8]:
import pandas as pd

In [9]:
df_results = pd.read_excel("results_with_original_text.xlsx")

In [10]:
df_results.set_index("file_name", inplace=True)

In [11]:
df_results

,text,elapsed_time,corrected_text,elapsed_time_corrected,Unnamed: 5,Unnamed: 6,Unnamed: 7,original_text
file_name,,,,,,,,
1,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nПригодна пить всем,5.1053,ВОДА СВЯТОЙ ИСТОЧНИК\n1 0Л\nПРИРОДНАЯ ПИТЬ ВСЕМ,0.0004,NaN,NaN,NaN,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nприродная питьевая...
2,"Вода Святой источник\n0,75л\nнегазированная вода",4.2579,ВОДА СВЯТОЙ ИСТОЧНИК\n0 75Л\nНЕГАЗИРОВАННАЯ ВОДА,0.0004,NaN,NaN,NaN,"Вода Святой источник\n0,75л\nнегазированная спорт"
3,Вода ШИШКИН ЛЕС\nпитьевая 1л\nАО «Заирoколина»,5.2103,ВОДА ШИШКИН ЛЕС\nПИТЬЕВАЯ 1Л\nАО ЗАИРOКОЛИНА,0.0006,NaN,NaN,NaN,Вода ШИШКИН ЛЕС\nпитьевая 1л\nнегазированная
4,Цена красная,3.4990,ЦЕНА КРАСНАЯ,0.0001,NaN,NaN,NaN,"Вода КРАСНАЯ ЦЕНА\n1,5л\nнегазированная"
5,Вода Святой Источник\n1.5л\nх.почечен. без сах...,5.0172,ВОДА СВЯТОЙ ИСТОЧНИК\n1 5Л\nХ ПЕЧЕН БЕЗ САХ ПИЛ Б,0.0005,NaN,NaN,NaN,"Вода СВЯТОЙ ИСТОЧНИК\n1,5л\nключевая, без газа..."
...,...,...,...,...,...,...,...,...
1004,Томаты,3.3057,ТОМАТЫ,0.0001,NaN,NaN,NaN,Томаты 1кг\nnan\nnan
1005,Ароматизированный,3.4513,ГЛАЗИРОВАННЫЙ,0.0003,NaN,NaN,NaN,Апельсины 1кг\nnan\nnan
1006,Нектарины\n1кг\nузбекские,4.2026,НЕКТАРИНЫ\n1КГ\nУЗБЕКСКИЕ,0.0005,NaN,NaN,NaN,Нектарины 1кг\nузбекские\nnan


In [12]:
df_results_corrector = df_results.copy()

In [13]:
df_results_corrector = df_results_corrector[["text", "corrected_text", "original_text"]]

In [14]:
df_results_corrector

,text,corrected_text,original_text
file_name,,,
1,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nПригодна пить всем,ВОДА СВЯТОЙ ИСТОЧНИК\n1 0Л\nПРИРОДНАЯ ПИТЬ ВСЕМ,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nприродная питьевая...
2,"Вода Святой источник\n0,75л\nнегазированная вода",ВОДА СВЯТОЙ ИСТОЧНИК\n0 75Л\nНЕГАЗИРОВАННАЯ ВОДА,"Вода Святой источник\n0,75л\nнегазированная спорт"
3,Вода ШИШКИН ЛЕС\nпитьевая 1л\nАО «Заирoколина»,ВОДА ШИШКИН ЛЕС\nПИТЬЕВАЯ 1Л\nАО ЗАИРOКОЛИНА,Вода ШИШКИН ЛЕС\nпитьевая 1л\nнегазированная
4,Цена красная,ЦЕНА КРАСНАЯ,"Вода КРАСНАЯ ЦЕНА\n1,5л\nнегазированная"
5,Вода Святой Источник\n1.5л\nх.почечен. без сах...,ВОДА СВЯТОЙ ИСТОЧНИК\n1 5Л\nХ ПЕЧЕН БЕЗ САХ ПИЛ Б,"Вода СВЯТОЙ ИСТОЧНИК\n1,5л\nключевая, без газа..."
...,...,...,...
1004,Томаты,ТОМАТЫ,Томаты 1кг\nnan\nnan
1005,Ароматизированный,ГЛАЗИРОВАННЫЙ,Апельсины 1кг\nnan\nnan
1006,Нектарины\n1кг\nузбекские,НЕКТАРИНЫ\n1КГ\nУЗБЕКСКИЕ,Нектарины 1кг\nузбекские\nnan


In [15]:
df_results_corrector["log"] = None
df_results_corrector["corrected_text"] = None

In [16]:
df_results_corrector

,text,corrected_text,original_text,log
file_name,,,,
1,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nПригодна пить всем,None,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nприродная питьевая...,None
2,"Вода Святой источник\n0,75л\nнегазированная вода",None,"Вода Святой источник\n0,75л\nнегазированная спорт",None
3,Вода ШИШКИН ЛЕС\nпитьевая 1л\nАО «Заирoколина»,None,Вода ШИШКИН ЛЕС\nпитьевая 1л\nнегазированная,None
4,Цена красная,None,"Вода КРАСНАЯ ЦЕНА\n1,5л\nнегазированная",None
5,Вода Святой Источник\n1.5л\nх.почечен. без сах...,None,"Вода СВЯТОЙ ИСТОЧНИК\n1,5л\nключевая, без газа...",None
...,...,...,...,...
1004,Томаты,None,Томаты 1кг\nnan\nnan,None
1005,Ароматизированный,None,Апельсины 1кг\nnan\nnan,None
1006,Нектарины\n1кг\nузбекские,None,Нектарины 1кг\nузбекские\nnan,None


In [17]:
df_results_corrector[df_results_corrector["text"].isna()]


,text,corrected_text,original_text,log
file_name,,,,
168,NaN,None,-\nnan\nnan,None
169,NaN,None,-\nnan\nnan,None
170,NaN,None,-\nnan\nnan,None
184,NaN,None,-\nnan\nnan,None
284,NaN,None,-\nnan\nnan,None
976,NaN,None,Киви 1кг\nnan\nnan,None


In [18]:
import json

In [20]:
def correct_text(multiline_text, corrector):
    txt_lines = str(multiline_text).split("\n")
    txt_other = []
    
    if len(txt_lines) > 2:
        txt_2lines = txt_lines[:2]
        txt_other = txt_lines[2:]
    else:
        txt_2lines = txt_lines
    corrected_lines = []
    corrected_logs = []
    for txt in txt_2lines:     
        corrected_text = correct_units(str(txt))   
        corrected_text, corrected_log = corrector.correct_text(corrected_text)

        corrected_lines.append(corrected_text)
        corrected_logs = corrected_logs + corrected_log
     
    corrected_lines = corrected_lines + txt_other
    corrected_text = "\n".join(corrected_lines)
    return corrected_text, corrected_logs


In [21]:
corrector = PhraseCorrectorNgrams("words.txt", score_cutoff=75, min_len=3)
for index, row in df_results_corrector.iterrows():
        corrected_text, corrected_logs = correct_text(row["text"], corrector)

        #if not isinstance(text, str):
        #    continue  # или можно что-то другое, напр. записать "None"
                
        df_results_corrector.at[index, "corrected_text"] = corrected_text
        df_results_corrector.at[index, "log"] = json.dumps(corrected_logs, ensure_ascii=False)


In [22]:
df_results_corrector[:100]

,text,corrected_text,original_text,log
file_name,,,,
1,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nПригодна пить всем,ВОДА СВЯТОЙ ИСТОЧНИК\n1.0Л\nПригодна пить всем,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nприродная питьевая...,"[{""position"": 0, ""original"": ""ВОДА"", ""correcte..."
2,"Вода Святой источник\n0,75л\nнегазированная вода","ВОДА СВЯТОЙ ИСТОЧНИК\n0,75Л\nнегазированная вода","Вода Святой источник\n0,75л\nнегазированная спорт","[{""position"": 0, ""original"": ""ВОДА"", ""correcte..."
3,Вода ШИШКИН ЛЕС\nпитьевая 1л\nАО «Заирoколина»,ВОДА ШИШКИН ЛЕСУ\nПИТЬЕВАЯ 1Л\nАО «Заирoколина»,Вода ШИШКИН ЛЕС\nпитьевая 1л\nнегазированная,"[{""position"": 0, ""original"": ""ВОДА"", ""correcte..."
4,Цена красная,ЦЕНА КРАСНАЯ,"Вода КРАСНАЯ ЦЕНА\n1,5л\nнегазированная","[{""position"": 0, ""original"": ""ЦЕНА"", ""correcte..."
5,Вода Святой Источник\n1.5л\nх.почечен. без сах...,ВОДА СВЯТОЙ ИСТОЧНИК\n1.5Л\nх.почечен. без сах...,"Вода СВЯТОЙ ИСТОЧНИК\n1,5л\nключевая, без газа...","[{""position"": 0, ""original"": ""ВОДА"", ""correcte..."
...,...,...,...,...
96,Водка ПЕТРОВ.\nРЕГЛАМЕНТ 0.25л\n40%,ВОДКА ПЕТРОВ.\nРЕГЛАМЕНТ 0.25Л\n40%,Водка ПЕТРОВ.\nРЕГЛАМЕНТ 0.25л\n0.4,"[{""position"": 0, ""original"": ""ВОДКА"", ""correct..."
97,Водка TUNDRA\n0.25л\nAUTHENTIC 40%,ВОДКА TUNDRA\n0.25Л\nAUTHENTIC 40%,Водка TUNDRA\n0.25л\nAUTHENTIC 40%,"[{""position"": 0, ""original"": ""ВОДКА"", ""correct..."
98,Водка ПЕТРОВ.\nРЕГЛАМЕНТ\n0.25л\n40%,ВОДКА ПЕТРОВ.\nРЕГЛАМЕНТ\n0.25л\n40%,Водка ПЕТРОВ.\nРЕГЛАМЕНТ 0.25л\n0.4,"[{""position"": 0, ""original"": ""ВОДКА"", ""correct..."


In [65]:
df_results_corrector.to_excel("results_with_corrected_text.xlsx", index=False)